In [4]:
import pandas as pd
import os
import csv
import numpy as np

In [9]:
def delete_data(data: str) -> None:
    df = pd.read_csv(data)
    cells_to_delete = int(df.size * 0.05)
    rows, cols = np.unravel_index(np.random.choice(df.size, cells_to_delete, replace=False), df.shape)
    df.iloc[rows, cols] = np.nan
    df.to_csv('../data/full_data/walmart_sales_with_NA.csv', encoding='utf-8', index=False)
    print("csv file has been successfully generated")

In [10]:
delete_data('../data/full_data/walmart_sales.csv')

/var/folders/x9/wfxfy30n21j10t1dqkkhfjjm0000gn/T/ipykernel_4330/1464362111.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.iloc[rows, cols] = np.nan


csv file has been successfully generated


In [11]:
import os

def split_file(input_file: str, output_dir: str, num_parts int) -> None:
    with open(input_file, 'r') as f:
        column_titles = f.readline().strip().split(',')
        content = f.readlines()[1:]  # Skip the header line and read the rest of the lines
        file_size = len(content)
        part_size = file_size // num_parts

        # Create directory to store files
        os.makedirs(output_dir, exist_ok=True)

        # Write each part into separate CSV files
        for i in range(num_parts):
            start_index = i * part_size
            end_index = start_index + part_size
            if i == num_parts - 1:  # Last part might be larger if file_size is not divisible by num_parts
                end_index = file_size
            
            part_content = content[start_index:end_index]
            part_filename = os.path.join(output_dir, f'test_part_{i + 1}.csv')
            with open(part_filename, 'w', newline='') as part_file:
                part_file.write(','.join(column_titles[:3] + column_titles[4:]) + '\n')  # Exclude 'Weekly_Sales' column
                
                # Write content row by row
                for line in part_content:
                    values = line.strip().split(',')
                    # Ensure that each row has the correct number of columns
                    corrected_values = values[:3] + values[4:]
                    part_file.write(','.join(corrected_values) + '\n')

    print(f'{num_parts} parts created successfully.')


split_file(input_file='../data/full_data/walmart_sales_with_NA.csv', output_dir='../airflow/dags/raw_data', num_parts=100)


100 parts created successfully.
